In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

In [4]:
class BlogState(TypedDict):
    topic: str
    outline: str
    content: str

In [6]:
def create_outline(state: BlogState) -> BlogState:
    
    # Extract the topic from the state
    topic = state["topic"]

    # Create a prompt for the model
    prompt = f"Create a detailed outline for a blog post about {topic}."

    # Get the outline from the model
    outline = model.invoke(prompt).content

    # Update the state with the generated outline
    state["outline"] = str(outline)

    return state

def create_blog_content(state: BlogState) -> BlogState:
    # Extract the outline from the state
    outline = state["outline"]

    # Create a prompt for the model
    prompt = f"Write a detailed blog post based on the following outline:\n{outline}"

    # Get the blog content from the model
    content = model.invoke(prompt).content

    # Update the state with the generated content
    state["content"] = str(content)

    return state

In [13]:
graph = StateGraph(BlogState)

# define the nodes
graph.add_node("create_outline",create_outline)
graph.add_node("create_blog_content",create_blog_content)
# define the edges

graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog_content")
graph.add_edge("create_blog_content", END)

# compile the graph
workflow = graph.compile()

In [ ]:
inital_state: BlogState = {"topic": "let var and const in javascript", "outline": "", "content": ""}
final_state = workflow.invoke(inital_state)
print(final_state)

In [ ]:
print(final_state["content"])

Here's a detailed outline for a blog post about `let`, `var`, and `const` in JavaScript, designed to be comprehensive and easy to understand for developers of varying experience levels.

---

## Blog Post Title Ideas:

*   The Modern JavaScript Variable Guide: Understanding `var`, `let`, and `const`
*   Decoding JavaScript Variables: When to Use `var`, `let`, and `const`
*   `var`, `let`, `const`: Your Ultimate Guide to Variable Declaration in JavaScript
*   Mastering JavaScript Scope: A Deep Dive into `var`, `let`, and `const`
*   Stop the Confusion: A Clear Explanation of `var`, `let`, and `const` in JS

---

## Detailed Blog Post Outline:

**Estimated Reading Time:** 8-12 minutes

---

### I. Introduction (Approx. 10% of total content)

*   **A. Catchy Hook:**
    *   "Remember the good old days of just `var`?" or "Confused by the different ways to declare variables in JavaScript?"
    *   Highlight the common confusion, especially for new developers or those coming from other langu